# Reflections dataset

In [1]:
def get_device():
    import torch
    return "cuda" if torch.cuda.is_available() else "cpu"

device = get_device()
print(f"Using {device} device")

Using cuda device


In [ ]:
batch_size = 10

In [3]:
def get_transforms():
    import torchvision.transforms as T
    
    return T.Compose(
        [T.ToTensor()]
    )

transforms = get_transforms()